# Label propagation baseline

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import random
import json
import pickle
import time

In [2]:
import torch
from torch_geometric.data import Data
from torch import nn
import sklearn
from sklearn.metrics import f1_score

In [3]:
# carpeta para guardar las predicciones
results_path = "./Predictions/"

# nombre de las predicciones
nombre_predicciones = 'Label_Propagation.pkl'

# carpeta donde se toman los datos
carpeta_datos = ".\\Data\\GNN_data\\"

## Cargar datos

In [4]:
# cargar los diccionarios de codificacion de id
with open(carpeta_datos + 'encode_mathid_2_nodeid.pkl', 'rb') as f:
    encode_mathid_2_nodeid = pickle.load(f)
    
with open(carpeta_datos + 'decode_nodeid_2_mathid.pkl', 'rb') as f:
    decode_nodeid_2_mathid = pickle.load(f)
    
print(f"Se tienen {len(encode_mathid_2_nodeid)} nodos")

Se tienen 267774 nodos


In [5]:
# cargar los diccionarios de codificacion de subject
with open(carpeta_datos + 'encode_subj_2_idx.pkl', 'rb') as f:
    encode_subj_2_idx = pickle.load(f)
    
with open(carpeta_datos + 'decode_idx_2_subj.pkl', 'rb') as f:
    decode_idx_2_subj = pickle.load(f)
    
# contar los subjects sin el unknown
numero_subjects = len(encode_subj_2_idx) - 1 # -1 por el unknown
    
print(f"Se tienen {numero_subjects} subjects (sin el unknown)")

Se tienen 63 subjects (sin el unknown)


In [6]:
# cargar edge index, y
edge_index = torch.load(carpeta_datos + 'edge_index.pt')
y =  torch.load(carpeta_datos + 'y.pt')

In [7]:
# cargar masks
train_mask = torch.load(carpeta_datos + 'train_mask.pt')
val_mask = torch.load(carpeta_datos + 'val_mask.pt')
test_mask = torch.load(carpeta_datos + 'test_mask.pt')
useless_mask = torch.load(carpeta_datos + 'useless_mask.pt')

In [8]:
# crear el objeto data
data = Data(y = y, edge_index = edge_index)

# poner el numero de nodos
data.num_nodes = y.shape[0]

# agregar los masks
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask
data.useless_mask = useless_mask

# ver que este bien
data.validate(raise_on_error=True)

from torch_geometric.utils import to_undirected

# convertir el grafo a no dirigido
data.edge_index = to_undirected(data.edge_index)

data

Data(edge_index=[2, 562545], y=[267774], num_nodes=267774, train_mask=[267774], val_mask=[267774], test_mask=[267774], useless_mask=[267774])

In [9]:
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of nodes: 267774
Number of edges: 562545
Average node degree: 2.10
Number of training nodes: 141345
Training node label rate: 0.53
Has isolated nodes: True
Has self-loops: False
Is undirected: False


## Funciones para evaluar

In [10]:
def compute_metrics(y_true, y_pred):
    '''
    Funcion que calcula estadisticas de unas predicciones
    '''

    # calcula f1 macro weighted y micro
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    f1_micro = f1_score(y_true, y_pred, average='micro')

    # poner las estadisticas en un diccionario
    dict_results = {
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted,
        'f1_micro': f1_micro,
    }

    return dict_results

In [11]:
def predecir_evaluar(model, mask):
    '''
    Dado un mask (puede ser train o val)
    Hace las predicciones, calcula estadisticas de estas
    '''
    
    # tomar el output del modelo
    out = model(y = data.y.to(torch.int64), edge_index = data.edge_index.to(torch.int64),
                mask = data.train_mask)  # denotando que solo se usa train para label propagation

    # tomar las predicciones, solo del mask correspondiente
    pred = out[mask].argmax(dim=-1, keepdim=False)

    # tomar las etiquetas reales para esos datos
    true = data.y[mask]

    # calcular estadisticas
    estadisticas = compute_metrics(true, pred)

    # devolver las estadisticas
    return estadisticas

In [12]:
def predicciones_full_data(model):
    '''
    Hace predicciones para todos los datos
    Se guardan en la ruta correespondiente, como un diccionario
    '''
    
    # tomar el output del modelo
    out = model(y = data.y.to(torch.int64), edge_index = data.edge_index.to(torch.int64),
                mask = data.train_mask) # denotando que solo se usa train para label propagation
    
    # tomar las predicciones, solo del mask correspondiente
    pred = out.argmax(dim=-1, keepdim=False)

    # poner las predicciones en formato de diccionario
    # math_id: prediccion
    predicciones_finales = dict()

    # recordar que se tiene un diccionario que nos da el math_id de cada indice
    # tambien se tiene un diccionario que da el subject de cada indice

    # iterar en las predicciones
    for idx_node, prediction in enumerate(pred.numpy()):

        # tomar el nodo del matematico
        math_id = decode_nodeid_2_mathid[idx_node]

        # tomar el subject
        subject = decode_idx_2_subj[prediction]

        # agregar la entrada
        predicciones_finales[math_id] =  subject
    # end for
    
    # guardar estas predicciones
    with open(results_path + nombre_predicciones, 'wb') as f:
        pickle.dump(predicciones_finales, f)

## Crear y correr el label progagation

In [13]:
from torch_geometric.nn import LabelPropagation

# crear el modelo, parametros del ejemplo
model = LabelPropagation(num_layers=3, alpha=0.9)

model

LabelPropagation(num_layers=3, alpha=0.9)

In [14]:
# ver desempeño del modelo entrenado en datos de train
print("Train data:")
train_metrics = predecir_evaluar(model, data.train_mask)
train_metrics

Train data:


{'f1_macro': 0.7853464566501729,
 'f1_weighted': 0.8591510795760021,
 'f1_micro': 0.8594502812267856}

In [15]:
# ver desempeño del modelo entrenado en datos de val
print("Val data:")
val_metrics = predecir_evaluar(model, data.val_mask)
val_metrics

Val data:


{'f1_macro': 0.48012809830788317,
 'f1_weighted': 0.6768460268381415,
 'f1_micro': 0.6569245463228272}

In [16]:
# hacer y guardar las predicciones en todos los datos
predicciones_full_data(model)